In [3]:
import sqlite3
import pandas as pd

In [4]:
pd.options.display.max_columns = None
#pd.options.display.max_rows = None

In [5]:
conn = sqlite3.connect('parch-and-posey.db')

In [6]:
cursor = conn.cursor()
cursor.execute('''
select * from sqlite_master where type = "table";
''')
columns = [col[0] for col in cursor.description]
data = cursor.fetchall()
cursor.close()

In [7]:
pd.DataFrame(data, columns=columns)

,type,name,tbl_name,rootpage,sql
0,table,web_events,web_events,2,"CREATE TABLE web_events (\tid integer,\taccoun..."
1,table,sales_reps,sales_reps,7,"CREATE TABLE sales_reps (\tid integer,\tname b..."
2,table,region,region,222,"CREATE TABLE region (\tid integer,\tname bpchar)"
3,table,orders,orders,223,"CREATE TABLE orders (\tid integer,\taccount_id..."
4,table,accounts,accounts,583,"CREATE TABLE accounts (\tid integer,\tname bpc..."


Task 1:

# Inequality JOINs

The query in Derek's video was pretty long. Let's now use a shorter query to showcase the power of joining with comparison operators.

Inequality operators (a.k.a. comparison operators) don't only need to be date times or numbers, they also work on strings! You'll see how this works by completing the following quiz, which will also reinforce the concept of joining with comparison operators.

In the following SQL Explorer, write a query that left joins the accounts table and the sales_reps tables on each sale rep's ID number and joins it using the < comparison operator on accounts.primary_poc and sales_reps.name, like so:

accounts.primary_poc < sales_reps.name

The query results should be a table with three columns: the account name (e.g. Johnson Controls), the primary contact name (e.g. Cammy Sosnowski), and the sales representative's name (e.g. Samuel Racine). Then answer the subsequent multiple choice question.

In [8]:
pd.read_sql_query(sql='''
SELECT *
FROM accounts a
JOIN sales_reps s
ON a.sales_rep_id = s.id AND
a.primary_poc < s.name
;
''', con=conn)

,id,name,website,lat,long,primary_poc,sales_rep_id,id,name,region_id
0,1021,Apple,www.apple.com,42.290495,-76.084009,Jodee Lupo,321520,321520,Michel Averette,1
1,1041,McKesson,www.mckesson.com,42.217093,-75.284998,Angeles Crusoe,321540,321540,Cara Clarke,1
2,1061,CVS Health,www.cvshealth.com,41.467796,-73.767636,Anabel Haskell,321560,321560,Elba Felder,1
3,1071,General Motors,www.gm.com,40.805518,-76.710181,Barrie Omeara,321570,321570,Shawanda Selke,1
4,1081,Ford Motor,www.ford.com,41.113942,-75.854225,Kym Hagerman,321580,321580,Sibyl Lauria,1
...,...,...,...,...,...,...,...,...,...,...
158,4411,O'Reilly Automotive,www.oreillyauto.com,45.531707,-122.645360,Chantell Drescher,321970,321970,Georgianna Chisholm,4
159,4431,Franklin Resources,www.franklinresources.com,45.533894,-122.682215,Dominique Favela,321970,321970,Georgianna Chisholm,4
160,4461,KKR,www.kkr.com,45.545353,-122.655247,Buffy Azure,321970,321970,Georgianna Chisholm,4
161,4471,Oneok,www.oneok.com,45.513513,-122.681500,Esta Engelhardt,321960,321960,Maryanna Fiorentino,4


In [13]:
pd.read_sql_query(sql='''
SELECT a.sales_rep_id AS a_sales_rep_id, s.id AS s_id,
a.primary_poc AS a_primary_poc, s.name AS s_name
FROM accounts a
LEFT JOIN sales_reps s
ON a_sales_rep_id = s_id AND
a_primary_poc < s_name
;
''', con=conn)

,a_sales_rep_id,s_id,a_primary_poc,s_name
0,321500,NaN,Tamara Tuma,None
1,321510,NaN,Sung Shields,None
2,321520,321520.0,Jodee Lupo,Michel Averette
3,321530,NaN,Serafina Banda,None
4,321540,321540.0,Angeles Crusoe,Cara Clarke
...,...,...,...,...
346,321970,321970.0,Buffy Azure,Georgianna Chisholm
347,321960,321960.0,Esta Engelhardt,Maryanna Fiorentino
348,321970,NaN,Khadijah Riemann,None
349,321960,321960.0,Deanne Hertlein,Maryanna Fiorentino


In [14]:
pd.read_sql_query(sql='''
SELECT COUNT(s_name) FROM(SELECT a.sales_rep_id AS a_sales_rep_id, s.id AS s_id,
a.primary_poc AS a_primary_poc, s.name AS s_name
FROM accounts a
LEFT JOIN sales_reps s
ON a_sales_rep_id = s_id AND
a_primary_poc < s_name)
;
''', con=conn)

,COUNT(s_name)
0,163


Solution:

SELECT accounts.name as account_name,
       accounts.primary_poc as poc_name,
       sales_reps.name as sales_rep_name
  FROM accounts
  LEFT JOIN sales_reps
    ON accounts.sales_rep_id = sales_reps.id
   AND accounts.primary_poc < sales_reps.name
   
For more details on how string comparison with < and > work in SQL, check out this excellent answer on Stack Overflow: SQL string comparison, greater than and less than operators:
https://stackoverflow.com/questions/26080187/sql-string-comparison-greater-than-and-less-than-operators/26080240#26080240